In [30]:
# Importando todos os componentes da biblioteca spark

import pyspark

In [31]:
#Importando as bases de dados sobre churn, as bases já vieram na proposta do projeto separadas entre treino e teste
DFtreino = spark.read.csv('data/projeto4_telecom_treino.csv', header = True, sep = ',', inferSchema = True)
DFteste = spark.read.csv('data/projeto4_telecom_teste.csv', header = True, sep = ',', inferSchema = True)

In [32]:
#Checando o nome das colunas, seus tipos e se essas são iguais em ambas as tabelas
DFtreino.printSchema() == DFteste.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- account_length: integer (nullable = true)
 |-- area_code: string (nullable = true)
 |-- international_plan: string (nullable = true)
 |-- voice_mail_plan: string (nullable = true)
 |-- number_vmail_messages: integer (nullable = true)
 |-- total_day_minutes: double (nullable = true)
 |-- total_day_calls: integer (nullable = true)
 |-- total_day_charge: double (nullable = true)
 |-- total_eve_minutes: double (nullable = true)
 |-- total_eve_calls: integer (nullable = true)
 |-- total_eve_charge: double (nullable = true)
 |-- total_night_minutes: double (nullable = true)
 |-- total_night_calls: integer (nullable = true)
 |-- total_night_charge: double (nullable = true)
 |-- total_intl_minutes: double (nullable = true)
 |-- total_intl_calls: integer (nullable = true)
 |-- total_intl_charge: double (nullable = true)
 |-- number_customer_service_calls: integer (nullable = true)
 |-- churn: string (nullable = t

True

In [33]:
#Analisando quantos % os dados de treino representam do total das bases 

DFtreino.count() / (DFtreino.count() + DFteste.count())

0.6666

In [34]:
#Após essa primeiras verificações, visualizarei a base em sí para verificar se os tipos atribuidos realmente fazem sentido 
# com os dados reais

DFtreino.show()


+---+-----+--------------+-------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+-----+
|_c0|state|account_length|    area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|churn|
+---+-----+--------------+-------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+--------------

In [38]:
# Para facilitar a visualização de forma separada, aqui é criada uma função para utilizar junto a um select, para selecionar
#apenas variaveis de certo tipo

def listVarByType(type):
    booleanVars = []

    for i in DFtreino.dtypes:
        if i[1] in f"{type}":
            booleanVars.append(i[0])
    return booleanVars

In [44]:
# Visualizando somente dados do tipo double

DFtreino.select(listVarByType('double')).show()



+-----------------+----------------+-----------------+----------------+-------------------+------------------+------------------+-----------------+
|total_day_minutes|total_day_charge|total_eve_minutes|total_eve_charge|total_night_minutes|total_night_charge|total_intl_minutes|total_intl_charge|
+-----------------+----------------+-----------------+----------------+-------------------+------------------+------------------+-----------------+
|            265.1|           45.07|            197.4|           16.78|              244.7|             11.01|              10.0|              2.7|
|            161.6|           27.47|            195.5|           16.62|              254.4|             11.45|              13.7|              3.7|
|            243.4|           41.38|            121.2|            10.3|              162.6|              7.32|              12.2|             3.29|
|            299.4|            50.9|             61.9|            5.26|              196.9|              8.86|  

In [42]:
# Visualizando somente dados do tipo integer

DFtreino.select(listVarByType('integer',)).show()

#Todas as variaveis parecem bater os tipos com os nomes dados, meno _c0 que parece ser uma especie de indice que pode se deletados

+---+--------------+---------------------+---------------+---------------+-----------------+----------------+-----------------------------+
|_c0|account_length|number_vmail_messages|total_day_calls|total_eve_calls|total_night_calls|total_intl_calls|number_customer_service_calls|
+---+--------------+---------------------+---------------+---------------+-----------------+----------------+-----------------------------+
|  1|           128|                   25|            110|             99|               91|               3|                            1|
|  2|           107|                   26|            123|            103|              103|               3|                            1|
|  3|           137|                    0|            114|            110|              104|               5|                            0|
|  4|            84|                    0|             71|             88|               89|               7|                            2|
|  5|            75|

In [47]:
#Deletando a coluna _c0

DFtreino = DFtreino.drop('_c0')

In [48]:
#Checando se a variavel foi deletada

DFtreino.select(listVarByType('integer')).show()

+--------------+---------------------+---------------+---------------+-----------------+----------------+-----------------------------+
|account_length|number_vmail_messages|total_day_calls|total_eve_calls|total_night_calls|total_intl_calls|number_customer_service_calls|
+--------------+---------------------+---------------+---------------+-----------------+----------------+-----------------------------+
|           128|                   25|            110|             99|               91|               3|                            1|
|           107|                   26|            123|            103|              103|               3|                            1|
|           137|                    0|            114|            110|              104|               5|                            0|
|            84|                    0|             71|             88|               89|               7|                            2|
|            75|                    0|          

In [91]:
#Uma vez que todos os dados double e integer são todos numéricos, podemos verificar a associação linear das variaveis
#através do coeficiente de correlação de pearson

from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *

#Usando vector assembler para relacionar as variaveis
assembler = VectorAssembler(inputCols = DFtreino.select(listVarByType('integer,double')).columns,
                           outputCol = "colunas",  handleInvalid = "keep")

Cordf = assembler.transform(DFtreino.select(listVarByType('integer,double'))).select("colunas")

#Calculando a relação entre cada variavel
correlacao = Correlation.corr(Cordf,"colunas","pearson").collect()[0][0]

#Transformando os valores da correlacao em linha
linhas = correlacao.toArray().tolist()

#Criando um df  spark com as linhas e o nome da coluna
Cordf = spark.createDataFrame(linhas,colunas)

#Devido ao menor tamanho da tabela de correlacao, transformando em um df pandas
Cordf = Cordf.toPandas()

#Criando uma nova coluna com os nomes das variaveis
Cordf['Variaveis'] = colunas

#Transformando as colunas criadas no index do df

Cordf.set_index('Variaveis')

#Com isso somos capazes de identificar alta colineariedade entra as variaveis total_night_charge e total_night_minute,
#total_eve_charge e total_eve_minute, total_day_charge e total_day_minutes

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
Variaveis,,,,,,,,,,,,,,,
account_length,1.000000,-0.004628,0.006216,0.038470,0.006214,-0.006757,0.019260,-0.006745,-0.008955,-0.013176,-0.008960,0.009514,0.020661,0.009546,-0.003796
number_vmail_messages,-0.004628,1.000000,0.000778,-0.009548,0.000776,0.017562,-0.005864,0.017578,0.007681,0.007123,0.007663,0.002856,0.013957,0.002884,-0.013263
total_day_minutes,0.006216,0.000778,1.000000,0.006750,1.000000,0.007043,0.015769,0.007029,0.004323,0.022972,0.004300,-0.010155,0.008033,-0.010092,-0.013423
total_day_calls,0.038470,-0.009548,0.006750,1.000000,0.006753,-0.021451,0.006462,-0.021449,0.022938,-0.019557,0.022927,0.021565,0.004574,0.021666,-0.018942
total_day_charge,0.006214,0.000776,1.000000,0.006753,1.000000,0.007050,0.015769,0.007036,0.004324,0.022972,0.004301,-0.010157,0.008032,-0.010094,-0.013427
total_eve_minutes,-0.006757,0.017562,0.007043,-0.021451,0.007050,1.000000,-0.011430,1.000000,-0.012584,0.007586,-0.012593,-0.011035,0.002541,-0.011067,-0.012985
total_eve_calls,0.019260,-0.005864,0.015769,0.006462,0.015769,-0.011430,1.000000,-0.011423,-0.002093,0.007710,-0.002056,0.008703,0.017434,0.008674,0.002423
total_eve_charge,-0.006745,0.017578,0.007029,-0.021449,0.007036,1.000000,-0.011423,1.000000,-0.012592,0.007596,-0.012601,-0.011043,0.002541,-0.011074,-0.012987
total_night_minutes,-0.008955,0.007681,0.004323,0.022938,0.004324,-0.012584,-0.002093,-0.012592,1.000000,0.011204,0.999999,-0.015207,-0.012353,-0.015180,-0.009288


In [92]:
#Após analisar as variaveis numéricas, podemos visualizar as variaveis texto

DFtreino.select(listVarByType('string')).show()

+-----+-------------+------------------+---------------+-----+
|state|    area_code|international_plan|voice_mail_plan|churn|
+-----+-------------+------------------+---------------+-----+
|   KS|area_code_415|                no|            yes|   no|
|   OH|area_code_415|                no|            yes|   no|
|   NJ|area_code_415|                no|             no|   no|
|   OH|area_code_408|               yes|             no|   no|
|   OK|area_code_415|               yes|             no|   no|
|   AL|area_code_510|               yes|             no|   no|
|   MA|area_code_510|                no|            yes|   no|
|   MO|area_code_415|               yes|             no|   no|
|   LA|area_code_408|                no|             no|   no|
|   WV|area_code_415|               yes|            yes|   no|
|   IN|area_code_415|                no|             no|  yes|
|   RI|area_code_415|                no|             no|   no|
|   IA|area_code_408|                no|             no

In [93]:
#Entre as variáveis strings, indentificamos três colunas onde os valores possíveis são somente Yes ou No
#Para facilitar a interpretação e treino do modelo, é possivel transforma-las em variaveis dummmys (0 ou 1)

from pyspark.sql.functions import when

DFtreino = DFtreino.withColumn(colName = "international_plan",col = when(DFtreino.international_plan == "no",0)\
                              .otherwise(1))\
           .withColumn(colName = 'voice_mail_plan', col =  when(DFtreino.voice_mail_plan == "no", 0)\
                       .otherwise(1))\
           .withColumn(colName = "churn", col = when(DFtreino.churn == "no",0)\
                      .otherwise(1))


In [94]:
#Verificando as novas variaveis alteradas, vale ressaltar que a variavel churn será a variavel dependente durante o desenvolvimento
#do modelo
DFtreino.select("international_plan","voice_mail_plan","churn").show()

+------------------+---------------+-----+
|international_plan|voice_mail_plan|churn|
+------------------+---------------+-----+
|                 0|              1|    0|
|                 0|              1|    0|
|                 0|              0|    0|
|                 1|              0|    0|
|                 1|              0|    0|
|                 1|              0|    0|
|                 0|              1|    0|
|                 1|              0|    0|
|                 0|              0|    0|
|                 1|              1|    0|
|                 0|              0|    1|
|                 0|              0|    0|
|                 0|              0|    0|
|                 0|              0|    0|
|                 0|              0|    0|
|                 0|              0|    1|
|                 0|              1|    0|
|                 0|              0|    0|
|                 0|              1|    0|
|                 0|              0|    0|
+----------

In [100]:
# Após transformações verificando o número de NaN, None ou Nulls de todas as colunas do dataframe
from pyspark.sql.functions import col, isnan

DFtreino.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in DFtreino.columns ]).show()

#É identificado a inexistencia de valores ausentes

+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+-----+
|state|account_length|area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|churn|
+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------------------